- load submission with iterative imp and reconstruct dataset with it - DONE

- for col in cols with missing values (or only F2_x and F4_x ?)
    - col is target
    - other cols are features
    - complete rows are for training
    - incomplete (with missing value) are for prediction
    - when the prediction is complete, update column values in dataset

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from tqdm import tqdm

# Load Data

In [2]:
data = pd.read_csv('../input/tabular-playground-series-jun-2022/data.csv', index_col = 'row_id')
submission = pd.read_csv('../input/tabular-playground-series-jun-2022/sample_submission.csv', index_col = 'row-col')
#sub_iter = pd.read_csv('../input/submission-iterativeimp/submission_iterative-imp.csv', index_col = 'row-col')
sub_iter = pd.read_csv('../input/submission-iterF4/submission_iterativeF_4.csv', index_col = 'row-col')

# display(data,submission,sub_iter)

# Apply results from Iterative Imputer

In [3]:
# use results from iterative imputation as a base for next steps -> fill values into data
data_imp = data.copy()
for i in tqdm(sub_iter.index):
    row = int(i.split('-')[0])
    col = i.split('-')[1]
    data_imp.at[row,col] = sub_iter.loc[i]

100%|██████████| 1000000/1000000 [01:42<00:00, 9782.72it/s]


# Treat cols with missing values as targets

In [4]:
%%time
 
from xgboost import XGBRegressor

model = XGBRegressor()

cols = data.columns[data.isnull().any()]
# start just with colums F_1_X
cols = [c for c in cols if c.startswith('F_1_')]
display(cols)

for col in tqdm(cols):
    # prepare dframes
    train_df = data_imp[data[col].notna()]
    test_df = data_imp[data[col].isna()]
    
    X_train = train_df.copy()
    y_train = X_train.pop(col)
    X_test = test_df.drop(col,axis=1)
    
#     display(X_train.columns, X_test.columns)
    
#     display(train_df,test_df)
#     display(X_train,y_train,X_test)
#     display(X_train,y_train,X_test)
    
    # model: train + predict
    model.fit(X_train,y_train)
    y = model.predict(X_test)
    
    # update df with new predictions
    data_imp.loc[data[col].isna(),col] = y
    

['F_1_0',
 'F_1_1',
 'F_1_2',
 'F_1_3',
 'F_1_4',
 'F_1_5',
 'F_1_6',
 'F_1_7',
 'F_1_8',
 'F_1_9',
 'F_1_10',
 'F_1_11',
 'F_1_12',
 'F_1_13',
 'F_1_14']

100%|██████████| 15/15 [3:09:13<00:00, 756.87s/it]

CPU times: user 12h 16min 35s, sys: 51.4 s, total: 12h 17min 26s
Wall time: 3h 9min 14s


# Submission

In [5]:
for i in tqdm(submission.index):
    row = int(i.split('-')[0])
    col = i.split('-')[1]
    submission.loc[i,'value'] = data_imp.loc[row,col]
    
    
submission.to_csv('submission.csv')

100%|██████████| 1000000/1000000 [01:25<00:00, 11742.24it/s]
